# Local Memory and Atomics

##### Sections
- [Local Memory Usage](#Local-Memory-Usage)
- _Code:_ [Local Memory Type and Size](#Local-Memory-Type-and-Size)
- [Local Accessors](#Local-Accessors)
- [Group Barrier](#Group-Barrier)
- _Code:_ [Matrix Multiplication without Local Memory](#Matrix-Multiplication-without-Local-Memory)
- _Code:_ [Matrix Multiplication with Local Memory](#Matrix-Multiplication-with-Local-Memory)
- [Atomic Operations](#Atomic-Operations)
- _Code:_ [Atomic Operations with Buffers](#Atomic-Operations-with-Buffers)
- _Code:_ [Atomic Operations with USM](#Atomic-Operations-with-USM)
- _Lab Exercise:_ [Atomic Operation](#Lab-Exercise:-Atomic-Operation)

## Learning Objectives
- Use local memory to avoid repeated global memory access
- Understand the usage of group barriers to synchronize all work-items
- Use atomic operation to perform reduction

# Local Memory Usage

Often work-items need to share data and communicate with each other. On one hand, all work-items in all work-groups can access global memory, so data sharing and communication can occur through global memory. However, due to its lower bandwidth and higher latency, sharing and communication through global memory is less efficient. On the other hand, work-items in a sub-group executing simultaneously in an execution unit (EU) thread can share data and communicate with each other very efficiently, but the number of work-items in a sub-group is usually small and the scope of data sharing and communication is very limited. 

Memory with higher bandwidth and lower latency accessible to a bigger scope of work-items is very desirable for data sharing communication among work-items. The shared local memory (SLM) in GPUs is designed for this purpose.

To simplify kernel development and accelerate communication between work-items in a work-group, SYCL defines a special local memory space specifically for communication between work-items in a work-group.

<img src="assets/localmem.png">

Each work-group may access variables in its own local memory space, but cannot access variables in another work-group’s local memory. When a work-group begins, the contents of its local memory are uninitialized, and local memory does not persist after a work-group finishes executing. Because of these properties, local memory may only be used for temporary storage while a work-group is executing.

For some devices, such as for many CPU devices, local memory is a software abstraction and is implemented using the same memory subsystems as global memory. On these devices, using local memory is primarily a convenience mechanism for communication. Some compilers may use the memory space information for compiler local memory its own local memory optimizations, but otherwise using local memory for communication will not fundamentally perform better than communication via global memory on these devices.

For other devices though, such as many GPU devices, there are dedicated resources for local memory, and on these devices, communicating via local memory will perform better than communicating via global memory.

We can use the device query `info::device::local_mem_type` to determine whether an accelerator has dedicated resources for local memory or whether local memory is implemented as a software abstraction of global memory. 

We can use the device query `info::device::local_mem_size` to determine the size of local memory available for each work-group to access.

### Local Memory Type and Size

The code below uses device query to determine the local memory size and type. Inspect code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [1]:
%%writefile lab/localmem_info.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  queue q;

  //# Print the device info
  std::cout << "device name   : " << q.get_device().get_info<info::device::name>() << "\n";
  std::cout << "local_mem_size: " << q.get_device().get_info<info::device::local_mem_size>() << "\n";

  auto local_mem_type = q.get_device().get_info<info::device::local_mem_type>();
  if(local_mem_type == info::local_mem_type::local) 
    std::cout << "local_mem_type: info::local_mem_type::local" << "\n";
  else if(local_mem_type == info::local_mem_type::global) 
    std::cout << "local_mem_type: info::local_mem_type::global" << "\n";
  else if(local_mem_type == info::local_mem_type::none) 
    std::cout << "local_mem_type: info::local_mem_type::none" << "\n";
 
  return 0;
}

Overwriting lab/localmem_info.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [2]:
! chmod 755 q; chmod 755 run_localmem_info.sh; if [ -x "$(command -v qsub)" ]; then ./q run_localmem_info.sh; else ./run_localmem_info.sh; fi

## u880bbe9dfaf42133c91b7f09426b221 is compiling SYCL_Essentials Module12 -- SYCL Atomics Local Memory - 3 of 5 localmem_info.cpp
device name   : Intel(R) Data Center GPU Max 1100
local_mem_size: 131072
local_mem_type: info::local_mem_type::local


### Local Accessors

A Local Accessor is used to declare local memory for use in an ND-range kernel. Like other accessor objects, a local accessor is constructed within a command group handler.

A local accessor is created by specifying a type and a range describing the number of elements of that type. Like other accessors, local accessors may be one-dimensional, two-dimensional, or three dimensional.

Below is an example of defining local_accessor `localmem` with type _int_ and _one-dimension_ data

```cpp
local_accessor<int, 1> localmem(N, h);
```

Below is an example of defining local_accessor `localmem` with type _float_ and _two-dimension_ data

```cpp
local_accessor<float, 2> localmem(range<2>(N, N), h);
```

The local accessor from one work-group can be accessed by all work-items within the work-group. Each work-group can have its own local accessor, work-item from another work-group cannot access this local accessor.

### Group Barrier

When local accessor data is shared, work-group barriers are often required for work-item synchronization.

The `group_barrier` function synchronizes how each work-item views the state of memory. This type of synchronization operation is known as enforcing memory consistency or fencing memory. It ensures that the results of memory operations performed before the barrier are visible to other work-items after the
barrier.

A `group_barrier` is usually required right after a local accessor is modified by a work-item so that it is synchronized for all work-items before the local accessor can be accessed.

Below is an example of how a `group_barrier` function is defined to synchronize across all work-items within the work-group:

```cpp
group_barrier(item.get_group());
```

## Local Memory Usage Example

When a computation requires repeated access to global memory data, using a local memory to load data from global memory and then accessing subsequent repeated access from local memory can be more performant.

One such example is matrix multiplication, multiplying two 8x8 matrices requires each of 8 rows to multiply with 8 columns, every row and column is accessed 8 times from global memory. 

<img src="assets/naive.PNG">

Using local memory for matrix multiplication can be more performant. Let's look at matrix multiplication without using local memory and using local memory to understand usage of `local accessor` and `group_barrier` concepts.

### Matrix Multiplication without Local Memory

The code below demonstrates basic matrix multiplication example. Inspect code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [3]:
%%writefile lab/matrixmul_16x16.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <sycl/sycl.hpp>
#include <iomanip>

using namespace sycl;

int main() {
    
    size_t N = 16;
    std::cout << "MATRIX_SIZE    : " << N << "x" << N << std::endl;

    //# Define vectors for matrices
    std::vector<float> matrix_a(N*N);
    std::vector<float> matrix_b(N*N);
    std::vector<float> matrix_c(N*N);
    std::vector<float> matrix_d(N*N);
    
    //# Initialize matrices with values
    float v1 = 2.f;
    float v2 = 3.f;
    for (int i=0; i<N; i++)
        for (int j=0; j<N; j++){
            matrix_a[i*N+j] = v1++;
            matrix_b[i*N+j] = v2++;
            matrix_c[i*N+j] = 0.f;
            matrix_d[i*N+j] = 0.f;
    }
    
    //# Define queue with default device for offloading computation
    queue q;
    std::cout << "Offload Device : " << q.get_device().get_info<info::device::name>() << std::endl;
    
    //# Create buffers for matrices
    buffer a(matrix_a);
    buffer b(matrix_b);
    buffer c(matrix_c);

    //# Submit command groups to execute on device
    q.submit([&](handler &h){
        //# Create accessors to copy buffers to the device
        accessor A(a, h, read_only);
        accessor B(b, h, read_only);
        accessor C(c, h, write_only);

        //# Define size for ND-range and work-group size
        range<2> global_size(N,N);
        range<2> work_group_size(N,N);

        //# Parallel Compute Matrix Multiplication
        h.parallel_for(nd_range<2>{global_size, work_group_size}, [=](nd_item<2> item){
            const int i = item.get_global_id(0);
            const int j = item.get_global_id(1);

            //# matrix multiplication computation from local memory
            float temp = 0.f;
            for (int k = 0; k < N; k++) {
                temp += A[i*N+k] * B[k*N+j];
            }
            C[i*N+j] = temp;
        });
    });
    host_accessor ha(c, read_only);
    
    //# Print Output and Verification
    auto FAIL = 0;
    for (int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            for(int k=0; k<N; k++){
                matrix_d[i*N+j] += matrix_a[i*N+k] * matrix_b[k*N+j];
            }
            if(matrix_d[i*N+j] != matrix_c[i*N+j]) FAIL = 1;
            std::cout << std::setw(6) << matrix_c[i*N+j] << " ";
        }
        std::cout << "\n";
    }
    if(FAIL == 1) std::cout << "FAIL\n"; else std::cout << "PASS\n";

    return 0;
}



Overwriting lab/matrixmul_16x16.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [4]:
! chmod 755 q; chmod 755 run_matrixmul_16x16.sh; if [ -x "$(command -v qsub)" ]; then ./q run_matrixmul_16x16.sh; else ./run_matrixmul_16x16.sh; fi

## u880bbe9dfaf42133c91b7f09426b221 is compiling SYCL_Essentials Module12 -- SYCL Atomics Local Memory - 4 of 5 matrixmul_16x16_localmem.cpp
MATRIX_SIZE    : 16x16
Offload Device : Intel(R) Data Center GPU Max 1100
 24136  24288  24440  24592  24744  24896  25048  25200  25352  25504  25656  25808  25960  26112  26264  26416 
 55624  56032  56440  56848  57256  57664  58072  58480  58888  59296  59704  60112  60520  60928  61336  61744 
 87112  87776  88440  89104  89768  90432  91096  91760  92424  93088  93752  94416  95080  95744  96408  97072 
118600 119520 120440 121360 122280 123200 124120 125040 125960 126880 127800 128720 129640 130560 131480 132400 
150088 151264 152440 153616 154792 155968 157144 158320 159496 160672 161848 163024 164200 165376 166552 167728 
181576 183008 184440 185872 187304 188736 190168 191600 193032 194464 195896 197328 198760 200192 201624 203056 
213064 214752 216440 218128 219816 221504 223192 224880 226568 228256 229944 231632 233320 235008 236696 23

### Matrix Multiplication with Local Memory

The code below demonstrates matrix multiplication example making use of local memory. Inspect code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [5]:
%%writefile lab/matrixmul_16x16_localmem.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <sycl/sycl.hpp>
#include <iomanip>

using namespace sycl;

int main() {
    
    size_t N = 16;
    std::cout << "MATRIX_SIZE    : " << N << "x" << N << std::endl;

    //# Define vectors for matrices
    std::vector<float> matrix_a(N*N);
    std::vector<float> matrix_b(N*N);
    std::vector<float> matrix_c(N*N);
    std::vector<float> matrix_d(N*N);
    
    //# Initialize matrices with values
    float v1 = 2.f;
    float v2 = 3.f;
    for (int i=0; i<N; i++)
        for (int j=0; j<N; j++){
            matrix_a[i*N+j] = v1++;
            matrix_b[i*N+j] = v2++;
            matrix_c[i*N+j] = 0.f;
            matrix_d[i*N+j] = 0.f;
    }
    
    //# Define queue with default device for offloading computation
    queue q;
    std::cout << "Offload Device : " << q.get_device().get_info<info::device::name>() << std::endl;
    
    //# Create buffers for matrices
    buffer a(matrix_a);
    buffer b(matrix_b);
    buffer c(matrix_c);

    //# Submit command groups to execute on device
    q.submit([&](handler &h){
        //# Create accessors to copy buffers to the device
        accessor A(a, h, read_only);
        accessor B(b, h, read_only);
        accessor C(c, h, write_only);

        //# Define size for ND-range and work-group size
        range<2> global_size(N,N);
        range<2> work_group_size(N,N);

        //# Create local accessors
        local_accessor<float, 2> A_local(range<2>(N, N), h);
        local_accessor<float, 2> B_local(range<2>(N, N), h);

        //# Parallel Compute Matrix Multiplication
        h.parallel_for(nd_range<2>{global_size, work_group_size}, [=](nd_item<2> item){
            const int i = item.get_global_id(0);
            const int j = item.get_global_id(1);
            const int x = item.get_local_id(0);
            const int y = item.get_local_id(1);

            //# copy from global to local memory
            A_local[x][y] = A[i * N + j];
            B_local[x][y] = B[i * N + j];

            //# barrier to sychronize local memory copy across all work items
            group_barrier(item.get_group());

            //# matrix multiplication computation from local memory
            float temp = 0.f;
            for (int k = 0; k < N; k++) {
                temp += A_local[x][k] * B_local[k][y];
            }
            C[i*N+j] = temp;
        });
    });
    host_accessor ha(c, read_only);
    
    //# Print Output and Verification
    auto FAIL = 0;
    for (int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            for(int k=0; k<N; k++){
                matrix_d[i*N+j] += matrix_a[i*N+k] * matrix_b[k*N+j];
            }
            if(matrix_d[i*N+j] != matrix_c[i*N+j]) FAIL = 1;
            std::cout << std::setw(6) << matrix_c[i*N+j] << " ";
        }
        std::cout << "\n";
    }
    if(FAIL == 1) std::cout << "FAIL\n"; else std::cout << "PASS\n";

    return 0;
}



Overwriting lab/matrixmul_16x16_localmem.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [6]:
! chmod 755 q; chmod 755 run_matrixmul_16x16_localmem.sh; if [ -x "$(command -v qsub)" ]; then ./q run_matrixmul_16x16_localmem.sh; else ./run_matrixmul_16x16_localmem.sh; fi

## u880bbe9dfaf42133c91b7f09426b221 is compiling SYCL_Essentials Module12 -- SYCL Atomics Local Memory - 5 of 5 matrixmul_16x16_localmem.cpp
MATRIX_SIZE    : 16x16
Offload Device : Intel(R) Data Center GPU Max 1100
 24136  24288  24440  24592  24744  24896  25048  25200  25352  25504  25656  25808  25960  26112  26264  26416 
 55624  56032  56440  56848  57256  57664  58072  58480  58888  59296  59704  60112  60520  60928  61336  61744 
 87112  87776  88440  89104  89768  90432  91096  91760  92424  93088  93752  94416  95080  95744  96408  97072 
118600 119520 120440 121360 122280 123200 124120 125040 125960 126880 127800 128720 129640 130560 131480 132400 
150088 151264 152440 153616 154792 155968 157144 158320 159496 160672 161848 163024 164200 165376 166552 167728 
181576 183008 184440 185872 187304 188736 190168 191600 193032 194464 195896 197328 198760 200192 201624 203056 
213064 214752 216440 218128 219816 221504 223192 224880 226568 228256 229944 231632 233320 235008 236696 23

# Atomic Operations

Atomic operations enable __concurrent access to a memory location without introducing a data race__. When multiple atomic operations access the same memory, they are guaranteed not to overlap. 

To understand why atomic operations are necessary, let look at few kernel examples to perform reduction, addition of N number of elements:

#### Serial Computation with single_task
A simple way to perform reduction is by using a for-loop to add all items in a single_task kernel submission as show below, but it does not take advantage of parallelism in hardware.
```cpp
     q.single_task([=](){
       for(int i=0; i<N; i++){
         sum[0] += data[i];
       }
     });
```

#### Parallel Computation with parallel_for may encounter race conditions
Using parallel_for for kernel submission will enable multiple work-items to execute concurrently but multiple work-item may try to update the same output variable causing __race conditions__.
```cpp
      q.parallel_for(N, [=](auto i) {
        sum[0] += data[i];
      });
```


#### Parallel Computation with atomic operation
The code snippet below show how to avoid race conditions when multiple work-items are trying to update the same memory location using atomic operations
```cpp
      q.parallel_for(N, [=](auto i) {
        auto atomic_var = atomic_ref<int, memory_order::relaxed, memory_scope::device, access::address_space::global_space>(sum[0]);

        atomic_var.fetch_add(data[i]);
      });
```

### atomic_ref class
The `atomic_ref` class above will make sure that the referenced variable will only be accessed atomically for the lifetime of the reference. It also specifies the _data type_, _memory order_ and _memory scope_.

```cpp
  auto atomic_var = atomic_ref<int, memory_order::relaxed, memory_scope::device, access::address_space::global_space>(result[0]);
```


#### memory_order
By providing the compiler with information about our desired memory order, we can prevent re-ordering optimizations that are incompatible with the intended behavior of our applications.
- `memory_order::relaxed`: Read and write operations can be re-ordered before or after the operation with no restrictions. There are no ordering guarantees.
- `memory_order::acquire`: Read and write operations appearing after the operation in the program must occur after it.
- `memory_order::release`:
Read and write operations appearing before the operation in the program must occur before it , and preceding write operations are guaranteed to be visible to other program instances which have been synchronized by a corresponding acquire operation.
- `memory_order::acq_rel`: 
The operation acts as both an acquire and a release. Read and write operations cannot be re-ordered around the operation, and preceding writes must be made visible as previously described for _memory_order::release_.
- `memory_order::seq_cst`: 
The operation acts as an acquire, release, or both depending on whether it is a read, write, or read-modify-write operation, respectively. All operations with this memory order are observed in a sequentially consistent order.

#### memory_scope
- `memory_scope::work_item`: The memory ordering constraint applies only to the calling work-item. This scope is only useful for image operations, as all other operations within a work-item are already guaranteed to execute in program order.
- `memory_scope::work_group`: The memory ordering constraint applies only to work-items in the same work-group as the calling work-item.
- `memory_scope::sub_group`: The memory ordering constraint applies only to work-items in the same sub-group as the calling work-item.
- `memory_scope::device`: The memory ordering constraint applies only to work-items executing on the same device as the calling work-item.
- `memory_scope::system`:  The memory ordering constraint applies to all work-items in the system.

### atomic operations

Atomic references to objects of integral and floating-point types extend the set of available atomic operations to include arithmetic operations

```cpp
        // integer and floating point
        atomic_var += data[i];         // addition
        atomic_var.fetch_add(data[i]); // addition
        atomic_var -= data[i];         // subtraction
        atomic_var.fetch_sub(data[i]); // subtraction
        atomic_var.fetch_max(data[i]); // maximum
        atomic_var.fetch_min(data[i]); // minimum
        // integer only
        atomic_var.fetch_and(data[i]); // bitwise AND
        atomic_var.fetch_or(data[i]);  // bitwise OR
        atomic_var.fetch_xor(data[i]); // bitwise XOR
```


### Atomic Operations with Buffers

The code below uses atomic operation to perform reduction with buffers memory model. Inspect code, there are no modifications necessary.

_[Note that using atomics to do reduction operation is not best approach, but it a easy example to demonstrate atomic operation functionality, for better performance with reduction can be achieved using SYCL reduction kernels]_

1. Inspect the code cell below and click run ▶ to save the code to file.

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [7]:
%%writefile lab/reduction_atomics_buffer.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

using namespace sycl;

static constexpr size_t N = 1024; // global size

int main() {
  queue q;
  std::cout << "Device : " << q.get_device().get_info<info::device::name>() << "\n";

  std::vector<int> data(N);
  for (int i = 0; i < N; i++) data[i] = i;
  int sum = 0;
  {
    //# create buffers for data and sum
    buffer buf_data(data);
    buffer buf_sum(&sum, range(1));

    //# Reduction Kernel using atomics 
    q.submit([&](auto &h) {
      accessor data_acc(buf_data, h, sycl::read_only);
      accessor sum_acc(buf_sum, h);

      h.parallel_for(N, [=](auto i) {
        auto sum_atomic = atomic_ref<int, 
          memory_order::relaxed, 
          memory_scope::device, 
          access::address_space::global_space>(sum_acc[0]);
        sum_atomic += data_acc[i];
      });
    });
  }
  std::cout << "Sum = " << sum << "\n";

  return 0;
}


Overwriting lab/reduction_atomics_buffer.cpp


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_reduction_atomics_buffer.sh; if [ -x "$(command -v qsub)" ]; then ./q run_reduction_atomics_buffer.sh; else ./run_reduction_atomics_buffer.sh; fi

### Atomic Operations with USM

The code below uses atomic operation to perform reduction with Unified Shared memory. Inspect code, there are no modifications necessary.

_[Note that using atomics to do reduction operation is not best approach, but it a easy example to demonstrate atomic operation functionality, for better performance with reduction can be achieved using SYCL reduction kernels]_

1. Inspect the code cell below and click run ▶ to save the code to file.

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/reduction_atomics_usm.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

using namespace sycl;

static constexpr size_t N = 1024; // global size

int main() {
  queue q;
  std::cout << "Device : " << q.get_device().get_info<info::device::name>() << "\n";

  auto data = malloc_shared<int>(N, q);
  for (int i = 0; i < N; i++) data[i] = i;
  auto sum = malloc_shared<int>(1, q);
  sum[0] = 0;

  //# Reduction Kernel using atomics 
  q.parallel_for(N, [=](auto i) {
    auto sum_atomic = atomic_ref<int, 
      memory_order::relaxed, 
      memory_scope::device, 
      access::address_space::global_space>(sum[0]);
    sum_atomic += data[i];
  }).wait();

  std::cout << "Sum = " << sum[0] << "\n";
  return 0;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_reduction_atomics_usm.sh; if [ -x "$(command -v qsub)" ]; then ./q run_reduction_atomics_usm.sh; else ./run_reduction_atomics_usm.sh; fi

## Lab Exercise: Atomic Operations

Complete the coding exercise below using Atomic Operations:
- The code has an array `data` of size `N=1024` elements initialized
- We will offload kernel task to find the minimum and maximum values from the `data` array using atomic operations
- Create atomic reference for minimum and maximum variables
- Create atomic operation in kernel to find minimum and maximum
- On the host, compute mid-range, which is average of min and max values

1. Edit the code cell below by following the steps and then click run ▶ to save the code to a file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/atomics_lab.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

using namespace sycl;

static constexpr size_t N = 1024; // global size

int main() {
  queue q;
  std::cout << "Device : " << q.get_device().get_info<info::device::name>() << "\n";

  auto data = malloc_shared<int>(N, q);
  for (int i = 0; i < N; i++) data[i] = i;
  auto min = malloc_shared<int>(1, q);
  auto max = malloc_shared<int>(1, q);
  min[0] = 0;
  max[0] = 0;

  //# Reduction Kernel using atomics 
  q.parallel_for(N, [=](auto i) {
    //# STEP 1: create atomic reference for min and max

    //# YOUR CODE GOES HERE
    
    
    
    
    //# STEP 2: add atomic operation for min and max computation  

    //# YOUR CODE GOES HERE
    
    
    
  }).wait();

  auto mid = 0.0;
  //# STEP 3: Compute mid-range using the min and max 

  //# YOUR CODE GOES HERE
    
    
    
  
  std::cout << "Minimum   = " << min[0] << "\n";
  std::cout << "Maximum   = " << max[0] << "\n";
  std::cout << "Mid-Range = " << mid << "\n";

  return 0;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_atomics_lab.sh; if [ -x "$(command -v qsub)" ]; then ./q run_atomics_lab.sh; else ./run_atomics_lab.sh; fi

***
# Summary

In this module you learned:
* How to setup and use Shared Local Memory in the device
* How to use atomic operation when using buffers or USM
